In [41]:
import requests
import pandas as pd
import json
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [42]:
#Marketing team wants to focus first on the best cities to travel to in France. According One Week In.com here are the top-35 cities to visit in France:
listofcities=["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

In [68]:
#This part allows the user to customize how we calculate the weather score, first checking if the user wants snow or not, and then if the user wants to change the default settings

C=False
while C==False:
    Y=input("Are you okay with snow? Y/N").lower()
    if Y=="y":
        snow=8
        C=True
    elif Y=='n':
        snow=-8
        C=True
    
C=False
while C==False:
    Y=input("Do you want to keep the default parameters for calculating weather score? Y/N").lower()
    if Y=="y":
        rain=50
        sun=1
        tempinput=1
        temp=tempinput*(snow-8)/-16
        weatherimportance=1
        C=True
    elif Y=='n':
        rain=float(input("Please enter importance of rain as a number, default is 50"))
        sun=float(input("Please enter importance of hours of sun as a number, default is 1"))
        if snow==-8:
            tempinput=float(input("Please enter importance of average temperature as a number, default is 1"))
            temp=tempinput*(snow-8)/-16
        else:
            temp=0
        weatherimportance=tempinput=float(input("Please enter importance of overall weather (ie cloudy) as a number, default is 1"))
        C=True


In [43]:
#Here we will scrap the latitudes and longitudes for the list of destination from open street map
listlat=[]
listlon=[]
for i in range(len(listofcities)):
    payload={"q":listofcities[i],"country":"FR", "format":"json"}
    r= requests.get(url="https://nominatim.openstreetmap.org/", params=payload)
    print("i is {}.r is {}. latitude is {} and longitude is {}".format(i,r,r.json()[0]["lat"],r.json()[0]["lon"]))
    listlat.append(r.json()[0]["lat"])
    listlon.append(r.json()[0]["lon"])

<Response [200]>
i is 0.r is <Response [200]>. latitude is 48.6359541 and longitude is -1.511459954959514
<Response [200]>
i is 1.r is <Response [200]>. latitude is 48.649518 and longitude is -2.0260409
<Response [200]>
i is 2.r is <Response [200]>. latitude is 49.2764624 and longitude is -0.7024738
<Response [200]>
i is 3.r is <Response [200]>. latitude is 49.4938975 and longitude is 0.1079732
<Response [200]>
i is 4.r is <Response [200]>. latitude is 49.4404591 and longitude is 1.0939658
<Response [200]>
i is 5.r is <Response [200]>. latitude is 48.8588897 and longitude is 2.3200410217200766
<Response [200]>
i is 6.r is <Response [200]>. latitude is 49.8941708 and longitude is 2.2956951
<Response [200]>
i is 7.r is <Response [200]>. latitude is 50.6365654 and longitude is 3.0635282
<Response [200]>
i is 8.r is <Response [200]>. latitude is 48.584614 and longitude is 7.7507127
<Response [200]>
i is 9.r is <Response [200]>. latitude is 48.249489800000006 and longitude is 7.344296202531

In [46]:
#We create a dataframe with the destination, lat and lon
dfcities = pd.DataFrame(
    {'Destination': listofcities,
     'Lat': listlat,
     'Lon': listlon
    })
dfcities = dfcities.astype({'Lat':'float'})
dfcities = dfcities.astype({'Lon':'float'})
dfcities.head()

In [48]:
#checking we still have 35 elements
len(dfcities)

35

In [49]:
#opening the keys file where our keys are stored. This file is not uploaded on github as the folder is public
dfkeys=pd.read_csv("keys.csv")

In [50]:
#we now scrape the weather details from open weather map
#creating empty lists to be used after
day=[]
averagefeltemperature=[]
percentagerain=[]
hoursofsun=[]
weathertype=[]
weathertypescore=[]
place=[]
#looping for each destination
for j in range(len(dfcities)):
    payload2={"units":"metric","appid":dfkeys["appid"][0],"exclude":"minutely,hourly","lat":round(dfcities["Lat"][j],2),"lon":round(dfcities["Lon"][j],2)}
    r2=requests.get(url="https://api.openweathermap.org/data/2.5/onecall", params=payload2)
    #Looping for each day of the next week (there is only daily data, not weekly)
    for i in range(7):
        day.append(i+1)
        place.append(dfcities["Destination"][j])
        #for the average temperature, middle day count for 50%, morning and evening for 25% each to give a better overall feeling of the day
        averagefeltemperature.append(round(r2.json()["daily"][i]["feels_like"]["day"]*0.5+r2.json()["daily"][i]["feels_like"]["eve"]*0.25+r2.json()["daily"][i]["feels_like"]["morn"]*0.25,2))
        percentagerain.append(r2.json()["daily"][i]["pop"])
        #calculating hours of sun (sunset-dawn) converted in hours
        hoursofsun.append(round((r2.json()["daily"][i]["sunset"]-r2.json()["daily"][i]["sunrise"])/3600,2))
        weathertype.append(r2.json()["daily"][i]["weather"][0]["description"])


In [51]:
#We store the data scraped into a dataframe
dfweather= pd.DataFrame(
    {"Destination":place,
        'AverageFeltTemperature': averagefeltemperature,
    "Day": day,
     'HoursOfSun': hoursofsun,
     'Percentageofchanceofrain': percentagerain,
     "WeatherType":weathertype
    })
dfweather.head(10)

In [53]:
#converting the weather type to a score, so we can use it later. Weather with snow will depends on choicies made by the user
dictweather={"thunderstorm with light rain":-3,"thunderstorm with rain":-5,"thunderstorm with heavy rain":-9,"light thunderstorm":1,"thunderstorm":-1,"heavy thunderstorm":-10,"ragged thunderstorm":-10,"thunderstorm with light drizzle":-2,"thunderstorm with drizzle":-3,"thunderstorm with heavy drizzle":-4,"light intensity drizzle":6,"drizzle":5,"heavy intensity drizzle":3,"light intensity drizzle rain":5,"drizzle rain":4,"heavy intensity drizzle rain":0,"shower rain and drizzle":snow,"heavy shower rain and drizzle":snow,"shower drizzle":1,"light rain":5,"moderate rain":3,"heavy intensity rain":1,"very heavy rain":0,"extreme rain":-4,"freezing rain":-4,"light intensity shower rain":4,"shower rain":3,"heavy intensity shower rain":0,"ragged shower rain":-2,"light snow":snow,"Snow":snow,"Heavy snow":-10,"Sleet":snow/2,"Light shower sleet":snow/2,"Shower sleet":snow/2,"Light rain and snow":snow/2,"Rain and snow":snow/2-3,"Light shower snow":snow/2,"Shower snow":snow,"Heavy shower snow":snow,"mist":5,"Smoke":-2,"Haze":3,"sand/ dust whirls":-10,"fog":2,"sand":-3,"dust":0,"volcanic ash":-50,"squalls":3,"tornado":-50,"clear sky":10,"few clouds":9,"scattered clouds":8,"broken clouds":7,"overcast clouds":6}
dfweather["WeatherTypeScore"]=dfweather["WeatherType"].map(dictweather)

In [70]:
#We calculate the weather score using either the defaults parameters or the customized one from the suer. Temperature is excluded (*0) if the user wants snow
dfweather["WeatherScore"]=dfweather["AverageFeltTemperature"]*temp+dfweather["Percentageofchanceofrain"]*rain+dfweather["HoursOfSun"]*sun+dfweather["WeatherTypeScore"]*weatherimportance


In [71]:
#we convert the dataframe from a daily view to a weekly view, taking averages
dfweatherweek=dfweather[["Destination","AverageFeltTemperature","HoursOfSun","Percentageofchanceofrain","WeatherTypeScore","WeatherScore"]].groupby("Destination").mean().round(2) 
dfweatherweek.head()

,AverageFeltTemperature,HoursOfSun,Percentageofchanceofrain,WeatherTypeScore,WeatherScore
Destination,,,,,
Aigues Mortes,18.50,14.31,0.12,7.29,68.26
Aix en Provence,18.95,14.31,0.17,7.00,72.00
Amiens,13.25,14.89,0.15,6.57,63.04
Annecy,14.26,14.51,0.68,5.29,99.29
Ariege,10.15,14.26,0.57,5.00,84.91


In [56]:
#We merge the weather table with the GPS coordinates, and create a destination ID for future database ease
dfcitieswithweather= pd.merge(dfcities,dfweatherweek,on="Destination")
dfcitieswithweather["DestinationID"]=dfcitieswithweather.index
dfcitieswithweather.head()

,Destination,Lat,Lon,AverageFeltTemperature,HoursOfSun,Percentageofchanceofrain,WeatherTypeScore,WeatherScore,DestinationID
0,Mont Saint Michel,48.635954,-1.511460,13.95,14.76,0.08,6.43,39.21,0
1,St Malo,48.649518,-2.026041,13.05,14.76,0.14,6.29,41.31,1
2,Bayeux,49.276462,-0.702474,12.67,14.83,0.15,6.14,41.07,2
3,Le Havre,49.493898,0.107973,12.40,14.85,0.12,6.86,40.11,3
4,Rouen,49.440459,1.093966,13.76,14.84,0.18,6.29,43.74,4


In [57]:
#We export it to a CSV
dfcitieswithweather.to_csv("Citieswithweather.csv")

In [58]:
#first graph, all destinations ranked by weather score
fig = px.scatter_mapbox(dfcitieswithweather, lat="Lat", lon="Lon", color="WeatherScore", size="WeatherScore", hover_name="Destination",hover_data={"WeatherScore":True,"AverageFeltTemperature":True,"HoursOfSun":True,"Percentageofchanceofrain":True,"WeatherTypeScore":True,"Lat":False,"Lon":False},zoom=4, mapbox_style="carto-positron", title="Map of 35 best destinations, ranked by best weather")
fig.show()

In [59]:
#second one withe only the top 5 as it was requested
fig = px.scatter_mapbox(dfcitieswithweather.sort_values("WeatherScore").tail(5), lat="Lat", lon="Lon", color="WeatherScore", size="WeatherScore", hover_name="Destination",hover_data={"WeatherScore":True,"AverageFeltTemperature":True,"HoursOfSun":True,"Percentageofchanceofrain":True,"WeatherTypeScore":True,"Lat":False,"Lon":False},zoom=4, mapbox_style="carto-positron", title="Map of 5 best destinations, ranked by best weather")
fig.show()